#### Candlestick Chart with Plotly

This work will provide basic candlestick visualization of apple share price dataset. Additionally, moving averages will be ploted in line with Bollinger Bands. 

Financial data is often plotted with a Japanese candlestick plot, so named because it was first created by 18th century Japanese rice traders. Such a chart can be created with matplotlib, though it requires considerable effort. However, plotly provides an opportunity to plot the candlestick chart easier. Bollinger Bands are a type of statistical chart characterizing the prices and volatility over time of a financial instrument or commodity, using a formulaic method propounded by John Bollinger in the 1980s. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

import quandl
quandl.ApiConfig.api_key = 'Insert Your API'

import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('your_login', 'Plotly_API')

In [2]:
start = datetime.datetime(2015,1,1)
end = datetime.date.today()

#as an example Mondelez share price dataset is chosen

mdlz = quandl.get("WIKI/" + "MDLZ", start_date=start, end_date=end)
mdlz.tail(3)

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2018-03-23,41.14,41.54,40.51,40.51,10917445.0,0.0,1.0,41.14,41.54,40.51,40.51,10917445.0
2018-03-26,41.06,41.40,40.68,41.33,7208515.0,0.0,1.0,41.06,41.40,40.68,41.33,7208515.0
2018-03-27,41.71,41.72,40.69,40.91,6570556.0,0.0,1.0,41.71,41.72,40.69,40.91,6570556.0


In [4]:
mdlz["14d"] = np.round(mdlz["Adj. Close"].rolling(window = 14, center = False).mean(), 3)
mdlz["20d"] = np.round(mdlz["Adj. Close"].rolling(window = 20, center = False).mean(), 3)
mdlz["200d"] = np.round(mdlz["Adj. Close"].rolling(window = 200, center = False).mean(), 3)

mdlz['20 Day STD'] = np.round(mdlz["Adj. Close"].rolling(window = 20, center = False).std(), 3)
mdlz['Upper Band'] = mdlz["20d"] + (mdlz['20 Day STD'] * 2)
mdlz['Lower Band'] = mdlz["20d"] - (mdlz['20 Day STD'] * 2)

In [5]:
#Candlestick chart with plotly
trace_1 = go.Candlestick(x=mdlz.index, open=mdlz.Open, name = 'GS', high=mdlz.High, low=mdlz.Low, 
                         close=mdlz.Close, increasing=dict(line=dict(color= '#17BECF')),
                       decreasing=dict(line=dict(color= '#7F7F7F')))

trace_2 = go.Scatter(
    x = mdlz.index,
    y = mdlz["20d"], name = '20d MA', mode = 'lines', line = dict(
        color = 'gold', width=1))
trace_3 = go.Scatter(
    x = mdlz.index, 
    y = mdlz["200d"], name = '200d MA', mode = 'lines', line = dict(
        color = ('rgb(22, 96, 167)'), shape='spline', width=1))
trace_4 = go.Scatter(
    x = mdlz.index, 
    y = mdlz['Upper Band'], legendgroup="Bollinger Bands", name = 'U-Band', mode = 'lines', line = dict(
        color = 'darkkhaki', shape='spline', width=.4))
trace_5 = go.Scatter(
    x = mdlz.index, 
    y = mdlz['Lower Band'],
  legendgroup="Bollinger Bands",name = 'L-Band',showlegend=True, mode = 'lines', line = dict(
        color = 'darkkhaki', shape='spline', width=.4))

data_plotly= [trace_1, trace_2, trace_3, trace_4, trace_5]
layout = go.Layout(title='Mondelez CandleStciks with Moving Averages',legend=dict(orientation="h"),
                   xaxis = dict(rangeslider = dict(visible = False)))


fig_plotly = go.Figure(data=data_plotly,layout=layout)
py.iplot(fig_plotly, filename='candlestick with MAs')